In [1]:
import pandas as pd
import numpy as np
from sklearn.compose import *
from sklearn.pipeline import *
from sklearn.preprocessing import FunctionTransformer
from sklearn.metrics import *
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score, GridSearchCV
from sklearn.impute import SimpleImputer
from category_encoders import CatBoostEncoder

In [2]:
# Load training data
X_train, y_train = pd.read_csv('https://github.com/noahmatsuyoshi/forest-cover-type-data/raw/master/X_train_big.csv'), pd.read_csv('https://github.com/noahmatsuyoshi/forest-cover-type-data/raw/master/y_train_big.csv')
X_train = X_train.drop('Unnamed: 0', axis=1)
y_train = y_train.drop('Unnamed: 0', axis=1).values.ravel()

In [3]:
# build column lists for ColumnTransformer
cat_columns = [f"Wilderness_Area{i}" for i in range(1,5)]
cat_columns += [f"Soil_Type{i}" for i in range(1,41)]
num_columns = ["Elevation", "Slope", 
               "Horizontal_Distance_To_Roadways", "Hillshade_9am", "Hillshade_Noon", "Hillshade_3pm", 
               "Horizontal_Distance_To_Fire_Points"]
hydro_columns = ["Horizontal_Distance_To_Hydrology", "Vertical_Distance_To_Hydrology"]
aspect_column = ["Aspect"]

In [4]:
# define simple functions for FunctionTransformers
def getDistance(x):
    return pd.DataFrame((
        x.loc[:,'Horizontal_Distance_To_Hydrology']**2 + 
        x.loc[:,'Vertical_Distance_To_Hydrology']**2
    )**0.5)

def passthrough(x):
    return x

In [5]:
# get physical distance from water using horizontal and vertical distances
hydro_distance = FunctionTransformer(getDistance)
hydro_distance_union = FeatureUnion([
    ('hydro_distance', hydro_distance),
    ('original_distances', FunctionTransformer(passthrough))
])

In [6]:
# compute sin and cos for aspect angle
aspect_sin = FunctionTransformer(np.sin)
aspect_cos = FunctionTransformer(np.cos)
aspect_union = FeatureUnion([
    ('aspect_sin', aspect_sin),
    ('aspect_cos', aspect_cos),
])

In [7]:
# build pipelines for each column list
hydro_pipe = Pipeline([
    ('impute', SimpleImputer(strategy='mean')),
    ('hydro_distance_union', hydro_distance_union),
])

aspect_pipe = Pipeline([
    ('impute', SimpleImputer(strategy='mean')),
    ('aspect_union', aspect_union),
])

cat_pipe = Pipeline([
    ('impute', SimpleImputer(strategy='most_frequent')),
    ('encoder', CatBoostEncoder())
])

num_pipe = Pipeline([
    ('impute', SimpleImputer(strategy='mean')),
])

In [8]:
# Build column transformer and RandomForestClassifier pipeline
col_transform = ColumnTransformer([
    ('cat', cat_pipe, cat_columns),
    ('num', num_pipe, num_columns),
    ('hydro_distance_union', hydro_distance_union, hydro_columns),
    ('aspect', aspect_union, aspect_column),
])

pipe_rf = Pipeline([
    ('col_transform', col_transform),
    ('model', RandomForestClassifier()),
])

pipe_gbc = Pipeline([
    ('col_transform', col_transform),
    ('model', GradientBoostingClassifier()),
])

In [9]:
# grid search for RandomForestClassifier
param_grid_rf = {
    'model__n_estimators': [100, 300],
    'model__min_samples_leaf': [1],
    'model__max_features': ['auto', 15, 30],
    'model__max_samples': [None, 5000],
}

param_grid_gbc = {
    'model__n_estimators': [100, 300],
    'model__min_samples_leaf': [1, 5],
    'model__max_features': ['auto', 15],
}

grid_search_rf = GridSearchCV(pipe_rf, param_grid_rf, n_jobs=-1, cv=StratifiedKFold(n_splits=5))
grid_search_rf.fit(X_train, y_train)

grid_search_gbc = GridSearchCV(pipe_gbc, param_grid_gbc, n_jobs=-1, cv=StratifiedKFold(n_splits=5))
grid_search_gbc.fit(X_train, y_train)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=False),
             estimator=Pipeline(steps=[('col_transform',
                                        ColumnTransformer(transformers=[('cat',
                                                                         Pipeline(steps=[('impute',
                                                                                          SimpleImputer(strategy='most_frequent')),
                                                                                         ('encoder',
                                                                                          CatBoostEncoder())]),
                                                                         ['Wilderness_Area1',
                                                                          'Wilderness_Area2',
                                                                          'Wilderness_Area3',
                                                         

In [10]:
# print grid search results
print(grid_search_rf.best_score_)
print(grid_search_rf.best_params_)


0.9638109447806398
{'model__max_features': 30, 'model__max_samples': None, 'model__min_samples_leaf': 1, 'model__n_estimators': 300}


In [11]:
# Build final pipeline
pipe = Pipeline([
    ('col_transform', col_transform),
    ('model', RandomForestClassifier(n_estimators=300, n_jobs=-1, class_weight='balanced')),
])

In [12]:
# Build feature name list after transformation 
# and show list of features and their importances
pipe.fit(X_train, y_train)
feature_importances = pipe._final_estimator.feature_importances_
column_names = cat_columns + num_columns + \
    ['hydro_distance'] + hydro_columns + \
    ['aspect_sin', 'aspect_cos']
normalized_importance = feature_importances / max(feature_importances)
sorted(list(zip(column_names, normalized_importance)), key=lambda t: t[1], reverse=True)

[('Elevation', 1.0),
 ('Horizontal_Distance_To_Roadways', 0.41796832065686695),
 ('Horizontal_Distance_To_Fire_Points', 0.34741395255788143),
 ('hydro_distance', 0.2108377600362886),
 ('Hillshade_9am', 0.19873441172652434),
 ('Wilderness_Area4', 0.1916785262692151),
 ('Vertical_Distance_To_Hydrology', 0.17935855787613394),
 ('Horizontal_Distance_To_Hydrology', 0.17254936940097002),
 ('Hillshade_3pm', 0.16899128965232674),
 ('Hillshade_Noon', 0.16631525019505133),
 ('Slope', 0.11570551428422465),
 ('Soil_Type10', 0.09442685208113773),
 ('Soil_Type3', 0.08675247916664719),
 ('Soil_Type38', 0.07874686176717265),
 ('Wilderness_Area1', 0.07321757525027896),
 ('Soil_Type39', 0.0730735645701963),
 ('Wilderness_Area3', 0.07239408454526515),
 ('aspect_cos', 0.0612021909432148),
 ('aspect_sin', 0.06119186384665129),
 ('Soil_Type4', 0.051918148203455576),
 ('Soil_Type40', 0.03729635338632367),
 ('Soil_Type30', 0.03276516580990188),
 ('Soil_Type2', 0.030054050183114942),
 ('Soil_Type22', 0.0249601

In [13]:
# use f1 weighted because this is a multiclass problem and the classes are imbalanced
scores = cross_val_score(pipe, X_train, y_train, cv=StratifiedKFold(n_splits=5), scoring="f1_weighted", n_jobs=-1)
scores

array([0.94606008, 0.94528701, 0.94509172, 0.94636801, 0.94686057])

In [14]:
# use balanced accuracy since the classes are imbalanced
scores = cross_val_score(pipe, X_train, y_train, cv=StratifiedKFold(n_splits=5), scoring="balanced_accuracy", n_jobs=-1)
scores

array([0.89433247, 0.88333142, 0.88417783, 0.88991936, 0.89108761])

In [15]:
# compute cv mean score
scores.mean()

0.8885697386484045

In [16]:
# compute training weighted f1 score
pipe.fit(X_train, y_train)
train_pred = pipe.predict(X_train)
f1_score(y_train, train_pred, average='weighted')

1.0

In [17]:
# show final pipeline properties
pipe

Pipeline(steps=[('col_transform',
                 ColumnTransformer(transformers=[('cat',
                                                  Pipeline(steps=[('impute',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   CatBoostEncoder())]),
                                                  ['Wilderness_Area1',
                                                   'Wilderness_Area2',
                                                   'Wilderness_Area3',
                                                   'Wilderness_Area4',
                                                   'Soil_Type1', 'Soil_Type2',
                                                   'Soil_Type3', 'Soil_Type4',
                                                   'Soil_Type5', 'Soil_Type6',
                                   

In [18]:
# run final pipeline on test set
X_test, y_test = pd.read_csv('https://github.com/noahmatsuyoshi/forest-cover-type-data/raw/master/X_test_big.csv'), pd.read_csv('https://github.com/noahmatsuyoshi/forest-cover-type-data/raw/master/y_test_big.csv')
X_test = X_test.drop('Unnamed: 0', axis=1)
y_test = y_test.drop('Unnamed: 0', axis=1).values.ravel()
predictions = pipe.predict(X_test)
predictions

array([1, 1, 1, ..., 2, 4, 1], dtype=int64)

In [19]:
# get test set accuracy
balanced_accuracy_score(y_test, predictions)

0.899217531252137

In [20]:
# get test set weighted f1
f1_score(y_test, predictions, average='weighted')

0.9515554993735198